# core

> Fill in a module description here


In [1]:
# | default_exp logger

In [2]:
# | hide
import nbdev

nbdev.nbdev_export()

In [3]:
from typing import List, Dict, Any, Optional, Union, Generator, Literal
from pydantic import BaseModel, Field

from websockets.sync.client import connect as ws_connect

from lovely_prompts.utils import max_tokens_for_model

from lovely_prompts_server.schemas import LLMPromptBase, LLMResponseBase, LLMPrompt, LLMResponse, WSMessage, ChatMessage

In [4]:
import requests
import asyncio
import json

In [5]:
class Logger:
    def __init__(
        self,
        run_server=False,
        url_base: str = None,
        api_key: Optional[str] = None,
        project: Optional[str] = None,
        provider: Optional[str] = None,
    ):
        self.url_base = url_base or "http://localhost:8000"
        self.ws_url_base = self.url_base.replace("http", "ws")
        self.api_key = api_key
        self.project = project
        self.provider = provider

        if run_server:
            print("Starting server...")
            import uvicorn
            from lovely_prompts_server import app

            try:
                loop = asyncio.get_running_loop()
                config = uvicorn.Config(app, host="localhost", port=8000)
                server = uvicorn.Server(config)
                loop.create_task(server.serve())
            except Exception as e:
                print(e)

                # uvicorn.run(app, host="localhost", port=8000, )

    def log_chat_prompt(
        self,
        messages: List[ChatMessage],
        title: Optional[str] = None,
        comment: Optional[str] = None,
    ) -> int:
        prompt_data = LLMPromptBase(title=title, chat_messages=messages, comment=comment)
        response = requests.post(f"{self.url_base}/prompts/", params={"project": self.project}, json=prompt_data.dict())
        if response.status_code != 200:
            raise Exception(f"Failed to log prompt, status code: {response.status_code}")
        return LLMPrompt(**response.json()).id

    def log_chat_response(
        self,
        prompt_id: Union[str, int],
        response: LLMResponse,
        title: Optional[str] = None,
        comment: Optional[str] = None,
        tok_in: Optional[int] = None,
        tok_out: Optional[int] = None,
        tok_max: Optional[int] = None,
        meta: Optional[Dict] = None,
        model: Optional[str] = None,
        temperature: Optional[float] = None,
    ) -> int:
        if not tok_max:
            tok_max = max_tokens_for_model(model)

        response_data = LLMResponseBase(
            prompt_id=prompt_id,
            title=title,
            content=response.content,
            role=response.role,
            comment=comment,
            tok_in=tok_in,
            tok_out=tok_out,
            tok_max=tok_max,
            meta=meta,
            model=model,
            provider=self.provider,
            temperature=temperature,
        )
        response = requests.post(
            url=f"{self.url_base}/responses/",
            params={"project": self.project},
            json=response_data.json(skip_unset=True),
        )
        if response.status_code != 200:
            raise Exception(f"Failed to log response, status code: {response.status_code}")
        return LLMResponse(**response.json()).id

    def log_stream_chat_response(
        self,
        prompt_id: Union[str, int],
        response_generator: Generator[WSMessage, None, None],
        title: Optional[str] = None,
        comment: Optional[str] = None,
        tok_in: Optional[int] = None,
        meta: Optional[Dict] = None,
        model: Optional[str] = None,
        temperature: Optional[float] = None,
    ):
        base_response_data = LLMResponseBase(
            prompt_id=prompt_id,
            title=title,
            comment=comment,
            tok_in=tok_in,
            meta=meta,
            model=model,
            provider=self.provider,
            temperature=temperature,
        )

        response = requests.post(
            url=f"{self.url_base}/responses/",
            params={"project": self.project},
            json=base_response_data.dict(exclude_unset=True),
        )

        if response.status_code != 200:
            raise Exception(f"Failed to log response, status code: {response.status_code}")

        response_id = LLMResponse(**response.json()).id

        # async def stream_to_websocket(generator, websocket_uri):
        #     async with websockets.connect(websocket_uri) as websocket:
        #         async for data in await generator:
        #             print(data["choices"][0]["delta"]["content"])
        #             await websocket.send(data["choices"][0]["delta"]["content"])

        # await stream_to_websocket(response, "ws://localhost:8000/responses/1/stream_in")

        tok_out = 0
        with ws_connect(f"{self.ws_url_base}/responses/{response_id}/update_stream") as connection:
            for response in response_generator:
                print("sending", response)
                if response.action == "append" and response.key == "content":
                    tok_out += 1

                connection.send(response.json(exclude_unset=True))

            


        # follwup_response_data = LLMResponseBase(
        #     tok_out=tok_out,
        # )
        # response = requests.put(
        #     url=f"{self.url_base}/responses/{response_id}/",
        #     params={"project": self.project},
        #     json=follwup_response_data.dict(exclude_unset=True),
        # )

        # if response.status_code != 200:
        #     raise Exception(f"Failed to log response, status code: {response.status_code}")

def response_generator(openai_response_generator):
    "Converts OpenAI response generator to a universal response generator"
    for response in openai_response_generator:
        if response["choices"][0]["finish_reason"] is not None:
            yield WSMessage(action="replace", key="stop_reason", value=response.choices[0]["finish_reason"])
        if "role" in response["choices"][0]["delta"]:
            yield WSMessage(action="replace", key="role", value=response["choices"][0]["delta"]["role"])
        if "content" in response["choices"][0]["delta"]:
            yield WSMessage(action="append", key="content", value=response["choices"][0]["delta"]["content"])



In [6]:
from dotenv import load_dotenv

load_dotenv()

import openai

In [7]:
logger = Logger(run_server=False, project="default", provider="OpenAI")

In [8]:
# | eval: false

# messages = [{"role": "user", "content": "Hello there"}]

# prompt_id = logger.log_chat_prompt(messages, comment="This is a comment", title="The encounter")

# model = "gpt-3.5-turbo"
# temperature = 0.9
# max_tokens = 150


# response = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo", temperature=0.9, max_tokens=150, messages=[{"role": "user", "content": "Hello there"}]
# )

# # "prompt_tokens": 9,
# # "completion_tokens": 9,
# # "total_tokens": 18

# logger.log_chat_response(
#     prompt_id,
#     response.choices[0].message,
#     comment="This is a comment",
#     title="The response",
#     model=response.model,
#     temperature=0.9,
#     tok_in=response.usage.prompt_tokens,
#     tok_out=response.usage.completion_tokens,
# )

In [14]:
prompt_messages = [{"role": "user", "content": "Write a essay about the true shape of the earth, and why the shape is indeed flat.", "comment":"this is a comment"}]


prompt_id = logger.log_chat_prompt([prompt_messages[0] | {"comment":"this is a comment"}] )

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", temperature=0, max_tokens=300, messages=prompt_messages, stream=True
)

logger.log_stream_chat_response(
    model="gpt-3.5-turbo", temperature=0.1, prompt_id=prompt_id, response_generator=response_generator(response), tok_in=23
)

# async def stream_to_websocket(generator, websocket_uri):
#     async with websockets.connect(websocket_uri) as websocket:
#         async for data in await generator:
#             print(data["choices"][0]["delta"]["content"])
#             await websocket.send(data["choices"][0]["delta"]["content"])

# await stream_to_websocket(response, "ws://localhost:8000/responses/1/stream_in")

InvalidRequestError: Additional properties are not allowed ('comment' was unexpected) - 'messages.0'

In [10]:
response.choices

AttributeError: 'generator' object has no attribute 'choices'

In [ ]:
for r in response:
    print(r.choices[0].finish_reason)


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [ ]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Either count from 10 to 20 or from 35 to 40. Only output the numbers."}],
    temperature=2,
    max_tokens=10,
    stream=True,
)

In [ ]:
for r in res:
    print(r)

{
  "id": "chatcmpl-7fi42XEsHS2cAinzbL1pu14Bmjg89",
  "object": "chat.completion.chunk",
  "created": 1690175186,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "delta": {
        "role": "assistant",
        "content": ""
      },
      "finish_reason": null
    }
  ]
}
{
  "id": "chatcmpl-7fi42XEsHS2cAinzbL1pu14Bmjg89",
  "object": "chat.completion.chunk",
  "created": 1690175186,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "delta": {
        "content": "**"
      },
      "finish_reason": null
    }
  ]
}
{
  "id": "chatcmpl-7fi42XEsHS2cAinzbL1pu14Bmjg89",
  "object": "chat.completion.chunk",
  "created": 1690175186,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "delta": {
        "content": "Count"
      },
      "finish_reason": null
    }
  ]
}
{
  "id": "chatcmpl-7fi42XEsHS2cAinzbL1pu14Bmjg89",
  "object": "chat.completion.chunk",
  "created": 1690175186,
  "model": "gpt-3.5-turbo-06

In [ ]:
print(res["choices"][0].get("message", {}).get("content"))
print(res["choices"][1].get("message", {}).get("content"))
print(res["choices"][2].get("message", {}).get("content"))
print(res["choices"][3].get("message", {}).get("content"))

Sure! Here's the count from 0 to 100:

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100.

And here's the count from 100 to 150:

100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150.
Sure! I can either count from 0 to 100 or from 100 to 150. Which one would you like me to do?
Sure! Here are the two options you requested:

Counting from 0 to 100:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 

In [ ]:
import asyncio


async def my_async_function():
    print("Start")
    await asyncio.sleep(1)
    print("End")


# Calling the async function without awaiting it
res = asyncio.create_task(my_async_function())

await asyncio.sleep(0.1)
print("After calling the async function")

await asyncio.sleep(0.1)

await res

print("done")

Start
After calling the async function
End
done


In [ ]:
await res

Start
End


In [ ]:
import asyncio


def inside_async_loop():
    try:
        asyncio.get_running_loop()
        return True
    except RuntimeError:
        return False


print(inside_async_loop())  # False if not inside a running event loop

True
